In [1]:
import datasets

In [2]:
dataset = datasets.load_dataset(
            "ILSVRC/imagenet-1k", cache_dir="/teamspace/studios/this_studio/data"
        )

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/datasets/load.py:1491: FutureWarning: The repository for ILSVRC/imagenet-1k contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/ILSVRC/imagenet-1k
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Loading dataset shards:   0%|          | 0/257 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/25 [00:00<?, ?it/s]

In [3]:
from diffusers.models import AutoencoderKL

vae_model = "stabilityai/sd-vae-ft-ema" 
vae = AutoencoderKL.from_pretrained(vae_model, cache_dir="/teamspace/studios/this_studio/data")
vae.to("cuda")
vae.eval()

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/diffusers/models/transformers/transformer_2d.py:34: FutureWarning: `Transformer2DModelOutput` is deprecated and will be removed in version 1.0.0. Importing `Transformer2DModelOutput` from `diffusers.models.transformer_2d` is deprecated and this will be removed in a future version. Please use `from diffusers.models.modeling_outputs import Transformer2DModelOutput`, instead.
  deprecate("Transformer2DModelOutput", "1.0.0", deprecation_message)


AutoencoderKL(
  (encoder): Encoder(
    (conv_in): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (down_blocks): ModuleList(
      (0): DownEncoderBlock2D(
        (resnets): ModuleList(
          (0-1): 2 x ResnetBlock2D(
            (norm1): GroupNorm(32, 128, eps=1e-06, affine=True)
            (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (norm2): GroupNorm(32, 128, eps=1e-06, affine=True)
            (dropout): Dropout(p=0.0, inplace=False)
            (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (nonlinearity): SiLU()
          )
        )
        (downsamplers): ModuleList(
          (0): Downsample2D(
            (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(2, 2))
          )
        )
      )
      (1): DownEncoderBlock2D(
        (resnets): ModuleList(
          (0): ResnetBlock2D(
            (norm1): GroupNorm(32, 128, eps=1e-06, affine=True)
            (c

In [4]:
from diffusers.image_processor import VaeImageProcessor

vaeprocess = VaeImageProcessor(vae_latent_channels=4) 

In [13]:

import torchvision.transforms as transforms
import torch

DEFAULT_IMAGE_SIZE = 256

DEFAULT_TRANSFORM = transforms.Compose(
    [
        transforms.Resize((DEFAULT_IMAGE_SIZE, DEFAULT_IMAGE_SIZE)),
        transforms.ToTensor(),
    ]
)

def gpu_computation(examples):
    images = [DEFAULT_TRANSFORM(image.convert("RGB")) for image in examples["image"]]
    images = torch.stack(images)
    images = vaeprocess.preprocess(images)
    images = images.to(device="cuda", dtype=torch.float)
    with torch.no_grad():
        latents = vae.encode(images).latent_dist.sample()
    return {"latents": latents, "label_latent": examples["label"]}



In [14]:


updated_dataset = dataset.map(
        gpu_computation,
        batched=True,
        batch_size=64,
        remove_columns=dataset["train"].column_names,
    )

updated_dataset.save_to_disk("/teamspace/studios/this_studio/data/imagenet_latents")

# push to ðŸ¤— hub


Map:   0%|          | 0/1281167 [00:00<?, ? examples/s]